In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as ex
import bz2
import torch
import _pickle as pickle
import scipy.stats as stats
import scipy
from plotly.subplots import make_subplots
import plotly.io as pio
import sys, os
from statsmodels.stats.multitest import multipletests as multi_tests

# Global Variables

In [69]:
RESULT_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results"
REF_SAVE_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results/organized/REF"
TRANS_SAVE_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results/organized/TRANS"
REDUC_SAVE_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results/organized/REDUC"
METRICS = ["recon_error", "delta_rot"]
METRICS_RENAME = ["Re. error", "RMS"]
SUFFIX = {"Raw":"_RAW", "Frozen":"_F", "Trained":"_T"}


# Functions

In [149]:
def pJoin(p1, p2):
    return os.path.join(p1,p2)

def load(file_path:str):
    with bz2.BZ2File(file_path, "rb") as f:
        obj = pickle.load(f)
    return obj
def save(file:object, file_path:str):
    with bz2.BZ2File(file_path, "w") as f:
        pickle.dump(file, f)
def mean_confidence_interval(data, confidence=0.95):
    data = np.nan_to_num(data, 0)
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h
    
def plot_bar(results:dict, title="", x_axis_name="Models", y_axis_name="Percentage (%)", 
                    width=500, height=300, vertical=False,
                    add_hline=False, annotation_text="baseline", annotation_position="top"):

    if not vertical:
        fig = ex.bar(results, x="Models", y="Percentage", color="Metrics", opacity=0.8, barmode='group', text="Percentage", template="seaborn")
    else:
        fig = ex.bar(results, y="Models", x="Percentage", color="Metrics", opacity=0.8, barmode='group', text="Percentage", template="seaborn")

    if add_hline:
        if not vertical:
            fig.add_hline(y=400, x0=-0.5, x1=len(results["Models"])/4 - 0.5,
            line_width=2, line_dash="dash", line_color="black", opacity=0.6,
            annotation_text=annotation_text, annotation_position=annotation_position) 
        else:
            fig.add_vline(x=400, y0=-0.5, y1=len(results["Models"])/4 - 0.5,
            line_width=2, line_dash="dash", line_color="black", opacity=0.6,
            annotation_text=annotation_text, annotation_position=annotation_position) 
    
    fig.update_traces(texttemplate='%{text}%', textposition="outside",cliponaxis=False)

    fig.update_layout(
    #    title_text=title,
        width = width, height=height, 
        font_family="Serif", font_size=14, 
        margin_l=5, margin_t=40, margin_b=5, margin_r=5)
    if vertical:
        t = x_axis_name
        x_axis_name = y_axis_name
        y_axis_name = t

    if x_axis_name is not None:
        fig.update_xaxes(
            title_text=x_axis_name,
        )
    if y_axis_name is not None:
        fig.update_yaxes(
            title_text = y_axis_name,
        )
    return fig

def save_plot(plot:object, file_path="figure.svg"):
    plot.write_image(file_path)

def calc_diff(*results, multiplier=100, inverse=False):
    if len(results) < 2:
        raise ValueError("Need at least 2 arrays for comparison")
    if (len(results[0])) < 2:
        raise ValueError("The result array should contain results from each rig")
    if not inverse:
        comp_fn = lambda base,target: np.asarray([1 / (y / x) * multiplier for x,y in zip(base, target)], np.float64) 
    else:
        comp_fn = lambda base,target: np.asarray([(y / x) * multiplier for x,y in zip(base, target)], np.float64) 

    base = results[0]
    comparison = [[],[]]
    for target in results[1:]:
        comp_avg_over_rig = np.round(np.mean([comp_fn(b,t) for b,t in zip(base, target)]), decimals=2)
        pval_avg_over_rig = np.max(multi_tests([stats.kruskal(t,b)[1] for b,t in zip(base, target)])[1])
        comparison[0].append(comp_avg_over_rig)
        comparison[1].append(pval_avg_over_rig)
    return comparison

def calc_pvals(*results):
    null_hypothesis = {}
    for metric in METRICS:
        p_vals = [stats.kruskal(*[m[metric][i] for m in results])[1] for i in range(len(results[0][metric]))]
        adjusted_pvals = np.max(multi_tests(p_vals)[1])
        null_hypothesis[metric] = "{:0.3e}".format(adjusted_pvals)
    return null_hypothesis


# Showing Adv. error and Rot. error are not significant

In [96]:
comp_df = {"RBF-CAT":{},"RBF-IN":{}, "VAE-CAT":{}, "VAE-IN":{},"DEC-CAT":{}, "DEC-IN":{}}
f1 = load(pJoin(REF_SAVE_PATH,"AE+MoE.pbz2"))
for k in comp_df.keys():
    f2 = load(pJoin(REF_SAVE_PATH,k+"+MoE.pbz2"))
    for metric in ["adv_error"]:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])

In [104]:
file_name = "FE_vs_FC:adv_error"
keys = list(comp_df.keys())
x = [k for k in keys]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in ["adv_error"]]
df_formatted = dict(Models=x, Percentage=y, Metrics=["Adv. error"]*len(keys))
fig = plot_bar(results=df_formatted, title="Generation performance of FC-CAT compared to FE", 
        width=600, add_hline=False, annotation_text="")
save_plot(fig, "img/ref/"+file_name+".svg")
fig.show()

In [68]:
for k in keys:
    for metric in ["adv_error"]:
        comp_df[k][metric]= "{:0.3e}".format(comp_df[k][metric][1][0])
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/ref/"+file_name+".csv")

             RBF-CAT     RBF-IN    VAE-CAT     VAE-IN    DEC-CAT     DEC-IN
adv_error  9.165e-01  8.653e-02  9.352e-01  7.677e-01  3.935e-01  6.060e-07


# Generation performance

## MoE vs LSTM

In [4]:
file_names = [] 
for _, _, names in os.walk(REF_SAVE_PATH):
    file_names = names
print(file_names)


['AE+LSTM.pbz2', 'AE+MoE.pbz2', 'DEC-CAT+MoE.pbz2', 'DEC-IN+MoE.pbz2', 'RBF-CAT+LSTM.pbz2', 'RBF-CAT+MoE.pbz2', 'RBF-IN+LSTM.pbz2', 'RBF-IN+MoE.pbz2', 'VAE-CAT+LSTM.pbz2', 'VAE-CAT+MoE.pbz2', 'VAE-IN+LSTM.pbz2', 'VAE-IN+MoE.pbz2']


In [112]:
comp_df = {"AE":{}, "RBF-IN":{}, "RBF-CAT":{}, "VAE-IN":{}, "VAE-CAT":{}}
for k in comp_df.keys():
    f1 = load(pJoin(REF_SAVE_PATH,k+"+MoE.pbz2"))
    f2 = load(pJoin(REF_SAVE_PATH,k+"+LSTM.pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])

In [85]:
null_comp_df = {} 
keys = list(comp_df.keys())
res = [load(pJoin(REF_SAVE_PATH,k+"+MoE.pbz2")) for k in keys]
for k in range(len(keys)):
    for j in range(k+1,len(keys)):
        pvals = calc_pvals(res[k],res[j])      
        null_comp_df[keys[k] + "_vs_" + keys[j]] = pvals

null_comp_df["AE_vs_RBF-CAT_vs_VAE-CAT"] = calc_pvals(res[0],res[2],res[4])
null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
print(df)
df.to_csv("tables/ref/MoE_all.csv")

                         recon_error   delta_rot
AE_vs_RBF-IN               4.891e-41   1.326e-09
AE_vs_RBF-CAT              2.044e-02   9.705e-01
AE_vs_VAE-IN               3.594e-34   1.031e-63
AE_vs_VAE-CAT              6.163e-01   9.459e-02
RBF-IN_vs_RBF-CAT          2.952e-55   1.613e-10
RBF-IN_vs_VAE-IN           5.200e-04   8.972e-47
RBF-IN_vs_VAE-CAT          9.076e-43   2.731e-01
RBF-CAT_vs_VAE-IN          1.038e-46   9.870e-64
RBF-CAT_vs_VAE-CAT         2.612e-01   7.535e-02
VAE-IN_vs_VAE-CAT          8.447e-32   4.100e-63
AE_vs_RBF-CAT_vs_VAE-CAT   3.027e-03   1.362e-01
All                       6.428e-105  1.373e-102


In [17]:
null_hypothesis = {}
MoEs = [load(pJoin(REF_SAVE_PATH,k+"+MoE.pbz2")) for k in comp_df.keys()]
for metric in METRICS:
    p_stats = np.mean([
        stats.kruskal(
            MoEs[0][metric][i],
            MoEs[4][metric][i]
            # MoEs[2][metric][i],
            # MoEs[3][metric][i],
            # MoEs[4][metric][i]
            )[1] for i in range(5)
        ])
    null_hypothesis[metric] = p_stats

{'recon_error': 0.12346279147061798, 'adv_error': 0.40870249625747157, 'rot_error': 0.09015967834665307, 'delta_rot': 0.01891790721886579}


In [110]:
keys = list(comp_df.keys())
x = [k for k in keys for _ in range(2)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Generation performance of LSTM compared to MoE", 
        width=700, add_hline=False)
save_plot(fig, "img/ref/MoE_vs_LSTM.svg")
fig.show()

In [114]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= "{:0.3e}".format(comp_df[k][metric][1][0])
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
# comp_pd.to_csv("tables/ref/"+file_name+".csv")
comp_pd.to_csv("tables/ref/MoE_vs_LSTM.csv")

## FC-CAT vs FC-IN

In [115]:
comp_df = {"RBF":{}, "VAE":{}, "DEC":{}}
for k in comp_df.keys():
    f1 = load(pJoin(REF_SAVE_PATH,k+"-IN+MoE.pbz2"))
    f2 = load(pJoin(REF_SAVE_PATH,k+"-CAT+MoE.pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])


In [118]:
null_comp_df = {} 
keys = list(comp_df.keys())
res = [load(pJoin(REF_SAVE_PATH,k+"-CAT+MoE.pbz2")) for k in keys]
for k in range(len(keys)):
    for j in range(k+1,len(keys)):
        pvals = calc_pvals(res[k],res[j])      
        null_comp_df[keys[k] + "_vs_" + keys[j]] = pvals

null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
print(df)
df.to_csv("tables/ref/FC-CAT_all.csv")

           recon_error  delta_rot
RBF_vs_VAE   2.612e-01  7.535e-02
RBF_vs_DEC   2.274e-08  4.492e-01
VAE_vs_DEC   4.586e-06  9.163e-03
All          1.768e-08  7.476e-07


In [121]:
keys = list(comp_df.keys())
x = [k for k in keys for i in range(2)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Generation performance of FC-CAT compared to FC-IN", 
        width=600, add_hline=False, annotation_text="")
save_plot(fig, "img/ref/FC-IN_vs_FC-CAT.svg")
fig.show()

In [123]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= "{:0.3e}".format(comp_df[k][metric][1][0])

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/ref/FC-IN_vs_FC-CAT.csv")

## FE vs FC-CAT

In [124]:
comp_df = {"RBF-CAT":{}, "VAE-CAT":{}, "DEC-CAT":{}}
f1 = load(pJoin(REF_SAVE_PATH,"AE+MoE.pbz2"))
for k in comp_df.keys():
    f2 = load(pJoin(REF_SAVE_PATH,k+"+MoE.pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])

In [125]:
file_name = "FE_vs_FC-CAT"
keys = list(comp_df.keys())
x = [k for k in keys for i in range(2)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Generation performance of FC-CAT compared to FE", 
        width=600, add_hline=False, annotation_text="")
save_plot(fig, "img/ref/"+file_name+".svg")
fig.show()

In [126]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= "{:0.3e}".format(comp_df[k][metric][1][0])

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/ref/"+file_name+".csv")

               RBF-CAT    VAE-CAT    DEC-CAT
recon_error  2.044e-02  6.163e-01  4.890e-13
delta_rot    9.705e-01  9.459e-02  9.114e-01


# Transferred quality
## FE Transferred (frozen, trained) vs FE Reference 

In [4]:
file_names = [] 
for _, _, names in os.walk(TRANS_SAVE_PATH):
    file_names = names
print(file_names)

['AE+MoE_F.pbz2', 'AE+MoE_RAW.pbz2', 'AE+MoE_T.pbz2', 'DEC-CAT+MoE_F.pbz2', 'DEC-CAT+MoE_RAW.pbz2', 'DEC-CAT+MoE_T.pbz2', 'DEC-IN+MoE_F.pbz2', 'DEC-IN+MoE_RAW.pbz2', 'DEC-IN+MoE_T.pbz2', 'RBF-CAT+MoE_F.pbz2', 'RBF-CAT+MoE_RAW.pbz2', 'RBF-CAT+MoE_T.pbz2', 'RBF-IN+MoE_F.pbz2', 'RBF-IN+MoE_RAW.pbz2', 'RBF-IN+MoE_T.pbz2', 'VAE-CAT+MoE_F.pbz2', 'VAE-CAT+MoE_RAW.pbz2', 'VAE-CAT+MoE_T.pbz2', 'VAE-IN+MoE_F.pbz2', 'VAE-IN+MoE_RAW.pbz2', 'VAE-IN+MoE_T.pbz2']


In [127]:
comp_df = {"Raw":{}, "Frozen":{}, "Trained":{}}
base = "AE+MoE"
f1 = load(pJoin(REF_SAVE_PATH, base+".pbz2"))
for k in comp_df.keys():
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+suffix+".pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric][1:], f2[metric])

In [129]:
null_comp_df = {} 
keys = list(comp_df.keys())
res = [load(pJoin(TRANS_SAVE_PATH,base+SUFFIX[k]+".pbz2")) for k in keys]

for k in range(len(keys)):
    for j in range(k+1,len(keys)):
        pvals = calc_pvals(res[k],res[j])      
        null_comp_df[keys[k] + "_vs_" + keys[j]] = pvals

null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
print(df)
df.to_csv("tables/trans/ae_all.csv")

                  recon_error  delta_rot
Raw_vs_Frozen       3.898e-62  1.761e-03
Raw_vs_Trained      1.175e-12  2.094e-01
Frozen_vs_Trained   8.439e-01  1.118e-02
All                 1.481e-67  1.440e-06


In [131]:
file_name = "FE_RAW_F_T"
keys = list(comp_df.keys())
x = [k for k in keys for i in range(2)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Transferring quality of FE", 
        width=600, add_hline=False, annotation_text="Reference perf.")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

In [132]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= "{:0.3e}".format(comp_df[k][metric][1][0])

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/trans/"+file_name+".csv")

                   Raw     Frozen    Trained
recon_error  2.485e-75  1.495e-11  1.901e-13
delta_rot    2.399e-03  3.153e-01  3.736e-01


In [133]:
comp_df = {"Frozen":{}, "Trained":{}}
base = "AE+MoE"
f1 = load(pJoin(TRANS_SAVE_PATH, base+"_RAW.pbz2"))
for k in comp_df.keys():
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+suffix+".pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])

In [134]:
file_name = "FE_F_T"
keys = list(comp_df.keys())
x = [k for k in keys for i in range(2)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Performance improvment of FE", 
        width=600, add_hline=False, annotation_text="Cold perf.", annotation_position="bottom")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

In [135]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= "{:0.3e}".format(comp_df[k][metric][1][0])

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/trans/"+file_name+".csv")

                Frozen    Trained
recon_error  3.898e-62  1.175e-12
delta_rot    1.761e-03  2.094e-01


## FC
### RBF

In [136]:
comp_df_in = {"Raw":{}, "Frozen":{}, "Trained":{}}
comp_df_cat = {"Raw":{}, "Frozen":{}, "Trained":{}}
base = "RBF"
for k in comp_df_in.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-IN+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric][1:], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-CAT+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric][1:], f2[metric])


In [139]:
null_comp_df = {} 
keys = list(comp_df_in.keys())
m_keys = [base+"-IN+MoE"+SUFFIX[k] for k in keys] + [base+"-CAT+MoE"+SUFFIX[k] for k in keys]
res = [load(pJoin(TRANS_SAVE_PATH,k + ".pbz2")) for k in m_keys]

for k in range(len(m_keys)):
    for j in range(k+1,len(m_keys)):
        pvals = calc_pvals(res[k],res[j])
        k1 = m_keys[k].replace(base,"").replace("-IN","IN").replace("-CAT","CAT").replace("+MoE","")
        k2 = m_keys[j].replace(base,"").replace("-IN","IN").replace("-CAT","CAT").replace("+MoE","")

        null_comp_df[k1+"_vs_"+k2] = pvals

null_comp_df["IN"] = calc_pvals(res[0],res[1],res[2])
null_comp_df["CAT"] = calc_pvals(res[3],res[4],res[5])
null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
print(df)
df.to_csv("tables/trans/rbf_all.csv")

                  recon_error   delta_rot
IN_RAW_vs_IN_F      9.268e-01   1.057e-14
IN_RAW_vs_IN_T      4.814e-03   9.567e-40
IN_RAW_vs_CAT_RAW   2.275e-47   5.280e-01
IN_RAW_vs_CAT_F    1.975e-151   1.167e-01
IN_RAW_vs_CAT_T    9.892e-157   1.993e-01
IN_F_vs_IN_T        5.444e-01   4.567e-01
IN_F_vs_CAT_RAW     2.377e-27   2.229e-21
IN_F_vs_CAT_F      5.731e-128   4.728e-33
IN_F_vs_CAT_T      4.989e-133   7.670e-24
IN_T_vs_CAT_RAW     9.680e-22   4.144e-59
IN_T_vs_CAT_F      2.195e-141   1.741e-66
IN_T_vs_CAT_T      6.122e-148   1.349e-62
CAT_RAW_vs_CAT_F    2.462e-57   1.000e-03
CAT_RAW_vs_CAT_T    1.401e-57   4.072e-01
CAT_F_vs_CAT_T      1.284e-01   6.821e-01
IN                  5.928e-03   3.117e-40
CAT                 7.294e-75   2.873e-03
All                 0.000e+00  6.906e-106


In [151]:
file_name = "RBF_RAW_F_T"
keys = list(comp_df_in.keys())
x = ["RBF-IN_" + k for k in keys for i in range(2)] + ["RBF-CAT_" + k for k in keys for i in range(2)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Transferring quality of FC - RBF", vertical=True,
        width=600, height=500, add_hline=False, annotation_text="", )
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

In [156]:
for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= comp_df_in[k][metric]= "{:0.3e}".format(comp_df_in[k][metric][1][0])
        comp_df_cat[k][metric]= comp_df_cat[k][metric]= "{:0.3e}".format(comp_df_cat[k][metric][1][0])
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                   Raw     Frozen    Trained
recon_error  1.953e-09  1.638e-04  1.953e-02
delta_rot    2.672e-06  9.839e-01  6.162e-04
                   Raw     Frozen    Trained
recon_error  5.962e-85  8.172e-16  6.308e-34
delta_rot    5.448e-01  8.777e-01  1.460e-01


In [157]:
# Against RAW

comp_df_in = {"Frozen":{}, "Trained":{}}
comp_df_cat = {"Frozen":{}, "Trained":{}}
base = "RBF"
for k in comp_df_in.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-IN+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-CAT+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric], f2[metric])


In [164]:
file_name = "RBF_F_T"
keys = list(comp_df_in.keys())
x = ["RBF-IN_" + k for k in keys for i in range(2)] + ["RBF-CAT_" + k for k in keys for i in range(2)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Performance improvment of FC - RBF", vertical=False,
        width=700, add_hline=False, annotation_text="Cold perf.", annotation_position="top right")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

In [167]:
for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= comp_df_in[k][metric]= "{:0.3e}".format(comp_df_in[k][metric][1][0])
        comp_df_cat[k][metric]= comp_df_cat[k][metric]= "{:0.3e}".format(comp_df_cat[k][metric][1][0])
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                Frozen    Trained
recon_error  2.462e-57  1.401e-57
delta_rot    1.000e-03  4.072e-01


### VAE

In [168]:
comp_df_in = {"Raw":{}, "Frozen":{}, "Trained":{}}
comp_df_cat = {"Raw":{}, "Frozen":{}, "Trained":{}}
base = "VAE"
for k in comp_df_in.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-IN+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric][1:], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-CAT+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric][1:], f2[metric])

null_comp_df = {} 
keys = list(comp_df_in.keys())
m_keys = [base+"-IN+MoE"+SUFFIX[k] for k in keys] + [base+"-CAT+MoE"+SUFFIX[k] for k in keys]
res = [load(pJoin(TRANS_SAVE_PATH,k + ".pbz2")) for k in m_keys]

for k in range(len(m_keys)):
    for j in range(k+1,len(m_keys)):
        pvals = calc_pvals(res[k],res[j])
        k1 = m_keys[k].replace(base,"").replace("-IN","IN").replace("-CAT","CAT").replace("+MoE","")
        k2 = m_keys[j].replace(base,"").replace("-IN","IN").replace("-CAT","CAT").replace("+MoE","")

        null_comp_df[k1+"_vs_"+k2] = pvals

null_comp_df["IN"] = calc_pvals(res[0],res[1],res[2])
null_comp_df["CAT"] = calc_pvals(res[3],res[4],res[5])
null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
df.to_csv("tables/trans/vae_all.csv")

file_name = base+"_RAW_F_T"
keys = list(comp_df_in.keys())
x = [base+"-IN_" + k for k in keys for i in range(2)] + [base+"-CAT_" + k for k in keys for i in range(2)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Transferring quality of FC - "+base, vertical=True,
        width=600, height=500, add_hline=False, annotation_text="")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= comp_df_in[k][metric]= "{:0.3e}".format(comp_df_in[k][metric][1][0])
        comp_df_cat[k][metric]= comp_df_cat[k][metric]= "{:0.3e}".format(comp_df_cat[k][metric][1][0])
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                   Raw     Frozen    Trained
recon_error  2.330e-29  1.019e-06  1.404e-06
delta_rot    1.286e-50  6.695e-01  1.326e-01
                   Raw     Frozen    Trained
recon_error  3.645e-54  2.702e-12  1.796e-02
delta_rot    8.780e-41  1.316e-14  8.095e-02


In [169]:
# Against RAW
comp_df_in = {"Frozen":{}, "Trained":{}}
comp_df_cat = {"Frozen":{}, "Trained":{}}
base = "VAE"
for k in comp_df_in.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-IN+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-CAT+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric], f2[metric])

file_name = base+"_F_T"
keys = list(comp_df_in.keys())
x = [base+"-IN_" + k for k in keys for i in range(2)] + [base+"-CAT_" + k for k in keys for i in range(2)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Performance improvement of FC - "+base, vertical=False,
        width=700, add_hline=False, annotation_text="Cold perf.", annotation_position="top right")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= comp_df_in[k][metric]= "{:0.3e}".format(comp_df_in[k][metric][1][0])
        comp_df_cat[k][metric]= comp_df_cat[k][metric]= "{:0.3e}".format(comp_df_cat[k][metric][1][0])
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                Frozen     Trained
recon_error  9.559e-03   3.642e-30
delta_rot    1.062e-17  7.661e-132
                Frozen     Trained
recon_error  1.769e-42   1.685e-42
delta_rot    4.537e-33  7.970e-120


In [170]:
comp_df_in = {"Raw":{}, "Frozen":{}, "Trained":{}}
comp_df_cat = {"Raw":{}, "Frozen":{}, "Trained":{}}
base = "DEC"
for k in comp_df_in.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-IN+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric][1:], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-CAT+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric][1:], f2[metric])

null_comp_df = {} 
keys = list(comp_df_in.keys())
m_keys = [base+"-IN+MoE"+SUFFIX[k] for k in keys] + [base+"-CAT+MoE"+SUFFIX[k] for k in keys]
res = [load(pJoin(TRANS_SAVE_PATH,k + ".pbz2")) for k in m_keys]

for k in range(len(m_keys)):
    for j in range(k+1,len(m_keys)):
        pvals = calc_pvals(res[k],res[j])
        k1 = m_keys[k].replace(base,"").replace("-IN","IN").replace("-CAT","CAT").replace("+MoE","")
        k2 = m_keys[j].replace(base,"").replace("-IN","IN").replace("-CAT","CAT").replace("+MoE","")

        null_comp_df[k1+"_vs_"+k2] = pvals

null_comp_df["IN"] = calc_pvals(res[0],res[1],res[2])
null_comp_df["CAT"] = calc_pvals(res[3],res[4],res[5])
null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
df.to_csv("tables/trans/dec_all.csv")

file_name = base+"_RAW_F_T"
keys = list(comp_df_in.keys())
x = [base+"-IN_" + k for k in keys for i in range(2)] + [base+"-CAT_" + k for k in keys for i in range(2)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Transferring quality of FC - "+base, vertical=True,
        width=600, height=500, add_hline=False, annotation_text="")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= comp_df_in[k][metric]= "{:0.3e}".format(comp_df_in[k][metric][1][0])
        comp_df_cat[k][metric]= comp_df_cat[k][metric]= "{:0.3e}".format(comp_df_cat[k][metric][1][0])
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                   Raw     Frozen    Trained
recon_error  9.401e-09  7.854e-06  2.459e-02
delta_rot    1.799e-14  8.240e-05  9.922e-01
                   Raw     Frozen    Trained
recon_error  1.214e-05  6.948e-02  4.369e-05
delta_rot    7.851e-01  9.495e-03  9.929e-01


In [171]:
# Against RAW
comp_df_in = {"Frozen":{}, "Trained":{}}
comp_df_cat = {"Frozen":{}, "Trained":{}}
base = "DEC"
for k in comp_df_in.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-IN+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-CAT+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric], f2[metric])

file_name = base+"_F_T"
keys = list(comp_df_in.keys())
x = [base+"-IN_" + k for k in keys for i in range(2)] + [base+"-CAT_" + k for k in keys for i in range(2)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Performance improvement of FC - "+base, vertical=False,
        width=700, add_hline=False, annotation_text="Cold perf.", annotation_position="top right")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= comp_df_in[k][metric]= "{:0.3e}".format(comp_df_in[k][metric][1][0])
        comp_df_cat[k][metric]= comp_df_cat[k][metric]= "{:0.3e}".format(comp_df_cat[k][metric][1][0])
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                Frozen    Trained
recon_error  1.831e-01  5.844e-07
delta_rot    1.006e-12  4.650e-31
                Frozen    Trained
recon_error  2.875e-12  3.691e-33
delta_rot    5.639e-35  2.288e-15


## REDUC TRANSFERRING QUALITY

In [23]:
file_names = [] 
for _, _, names in os.walk(REDUC_SAVE_PATH):
    file_names = names
print(file_names)

['AE+MoE_F.pbz2', 'AE+MoE_T.pbz2', 'DEC-CAT+MoE_F.pbz2', 'DEC-CAT+MoE_T.pbz2', 'RBF-CAT+MoE_F.pbz2', 'RBF-CAT+MoE_T.pbz2', 'VAE-CAT+MoE_F.pbz2', 'VAE-CAT+MoE_T.pbz2']


In [180]:
comp_df_ae = {"Frozen":{}, "Trained":{}}
comp_df_rbf = {"Frozen":{}, "Trained":{}}
comp_df_vae = {"Frozen":{}, "Trained":{}}
comp_df_dec = {"Frozen":{}, "Trained":{}}

def fill_df(comp_df,base):
    for k in comp_df.keys():
        suffix = SUFFIX[k]
        if base == "AE":
            f1 = load(pJoin(REF_SAVE_PATH, base+"+MoE.pbz2"))
            f2 = load(pJoin(REDUC_SAVE_PATH,base+"+MoE"+suffix+".pbz2"))
        else:
            f1 = load(pJoin(REF_SAVE_PATH, base+"-CAT+MoE.pbz2"))
            f2 = load(pJoin(REDUC_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
            
        for metric in METRICS:
            comp_df[k][metric] = calc_diff(f1[metric][1:], f2[metric])

fill_df(comp_df_ae, "AE")
fill_df(comp_df_rbf, "RBF")
fill_df(comp_df_vae, "VAE")
fill_df(comp_df_dec, "DEC")



In [174]:
null_comp_df = {}
bases = ["AE", "RBF", "VAE", "DEC"] 
keys = list(comp_df_ae.keys())
m_keys =    ["AE+MoE"+SUFFIX[k] for k in keys] + \
            ["RBF-CAT+MoE"+SUFFIX[k] for k in keys] + \
            ["VAE-CAT+MoE"+SUFFIX[k] for k in keys] + \
            ["DEC-CAT+MoE"+SUFFIX[k] for k in keys]

res = [load(pJoin(REDUC_SAVE_PATH, k+".pbz2")) for k in m_keys]

for k in range(len(m_keys)):
    for j in range(k+1,len(m_keys)):
        pvals = calc_pvals(res[k],res[j])
        k1 = m_keys[k].replace("-CAT","").replace("+MoE","")
        k2 = m_keys[j].replace("-CAT","").replace("+MoE","")

        null_comp_df[k1+"_vs_"+k2] = pvals

null_comp_df["FC_Frozen"] = calc_pvals(res[2],res[4],res[6])
null_comp_df["FC_Trained"] = calc_pvals(res[3],res[5],res[7])
null_comp_df["Frozen"] = calc_pvals(res[0],res[2],res[4],res[6])
null_comp_df["Trained"] = calc_pvals(res[1],res[3],res[5],res[7])
null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
# print(df)
df.to_csv("tables/reduc/fs_all.csv")

In [177]:
file_name = "reduc_RAW_F_T"
keys = list(comp_df_ae.keys())
x = ["AE_" + k for k in keys for i in range(2)] + \
    ["RBF-CAT_" + k for k in keys for i in range(2)] + \
    ["VAE-CAT_" + k for k in keys for i in range(2)] + \
    ["DEC-CAT_" + k for k in keys for i in range(2)]

y = []
for k in keys:
    y += [comp_df_ae[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_rbf[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_vae[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_dec[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*4)
fig = plot_bar(results=df_formatted, title="Transferring quality of FS", vertical=True,
        width=600, height=600, add_hline=False, annotation_text="Reference perf.", annotation_position="top right")
save_plot(fig, "img/reduc/"+file_name+".svg")
fig.show()


In [181]:
for k in keys:
    for metric in METRICS:
        comp_df_ae[k][metric]= comp_df_ae[k][metric]= "{:0.3e}".format(comp_df_ae[k][metric][1][0])
        comp_df_rbf[k][metric]= comp_df_rbf[k][metric]= "{:0.3e}".format(comp_df_rbf[k][metric][1][0])
        comp_df_vae[k][metric]= comp_df_vae[k][metric]= "{:0.3e}".format(comp_df_vae[k][metric][1][0])
        comp_df_dec[k][metric]= comp_df_dec[k][metric]= "{:0.3e}".format(comp_df_dec[k][metric][1][0])

        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_ae = pd.DataFrame(comp_df_ae)
comp_pd_rbf = pd.DataFrame(comp_df_rbf)
comp_pd_vae = pd.DataFrame(comp_df_vae)
comp_pd_dec = pd.DataFrame(comp_df_dec)

print(comp_pd_ae)
print(comp_pd_rbf)
print(comp_pd_vae)
print(comp_pd_dec)
comp_pd_ae.to_csv("tables/reduc/"+file_name+"_ae.csv")
comp_pd_rbf.to_csv("tables/reduc/"+file_name+"_rbg.csv")
comp_pd_vae.to_csv("tables/reduc/"+file_name+"_vae.csv")
comp_pd_dec.to_csv("tables/reduc/"+file_name+"_dec.csv")

                Frozen    Trained
recon_error  6.516e-17  1.753e-14
delta_rot    2.989e-01  9.530e-02
                Frozen    Trained
recon_error  3.599e-81  4.943e-40
delta_rot    8.863e-01  8.887e-01
                Frozen    Trained
recon_error  1.752e-59  2.072e-15
delta_rot    4.526e-25  6.626e-01
                Frozen    Trained
recon_error  1.007e-07  5.757e-11
delta_rot    8.964e-01  4.217e-02


# Summary

In [182]:
comp_df_ae = {"AE_{FE}":{}, "AE_{FS}":{}}
comp_df_rbf = {"RBF_{FC}":{}, "RBF_{FS}":{}}
comp_df_vae = {"VAE_{FC}":{}, "VAE_{FS}":{}}
comp_df_dec = {"DEC_{FC}":{}, "DEC_{FS}":{}}

def fill_df(comp_df,base):
    for k in comp_df.keys():
        suffix = "_T"
        f1 = load(pJoin(REF_SAVE_PATH, "AE+MoE.pbz2"))
        path = REDUC_SAVE_PATH if "_{FS}" in k else TRANS_SAVE_PATH
        if base == "AE":
            f2 = load(pJoin(path,base+"+MoE"+suffix+".pbz2"))
        else:
            f2 = load(pJoin(path,base+"-CAT+MoE"+suffix+".pbz2"))
                
        for metric in METRICS:
            comp_df[k][metric] = calc_diff(f1[metric][1:], f2[metric])

fill_df(comp_df_ae, "AE")
fill_df(comp_df_rbf, "RBF")
fill_df(comp_df_vae, "VAE")
fill_df(comp_df_dec, "DEC")



In [183]:
null_comp_df = {}
bases = ["AE", "RBF", "VAE", "DEC"] 
keys =    ["AE+MoE_T","RBF-CAT+MoE_T","VAE-CAT+MoE_T","DEC-CAT+MoE_T"]
m_keys =    [
            "AE+MoE_FE", "RBF-CAT+MoE_FC", "VAE-CAT+MoE_FC", "DEC-CAT+MoE_FC",
            "AE+MoE_FS", "RBF-CAT+MoE_FS", "VAE-CAT+MoE_FS", "DEC-CAT+MoE_FS"
            ]

res = [load(pJoin(TRANS_SAVE_PATH, k+".pbz2")) for k in keys] + [load(pJoin(REDUC_SAVE_PATH, k+".pbz2")) for k in keys]

for k in range(len(m_keys)):
    for j in range(k+1,len(m_keys)):
        pvals = calc_pvals(res[k],res[j])
        k1 = m_keys[k].replace("-CAT","").replace("+MoE","")
        k2 = m_keys[j].replace("-CAT","").replace("+MoE","")
        null_comp_df[k1+"_vs_"+k2] = pvals

null_comp_df["FE_vs_FC"] = calc_pvals(res[0],res[1],res[3],res[3])
null_comp_df["FE_vs_FS"] = calc_pvals(res[0],res[5],res[6],res[7])
null_comp_df["FC_vs_FS"] = calc_pvals(res[4],res[5],res[6],res[7])
null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
print(df)
df.to_csv("tables/summary_all.csv")

                 recon_error  delta_rot
AE_FE_vs_RBF_FC    3.847e-12  5.973e-01
AE_FE_vs_VAE_FC    1.920e-01  4.673e-27
AE_FE_vs_DEC_FC    1.936e-03  1.804e-02
AE_FE_vs_AE_FS     4.904e-01  8.228e-01
AE_FE_vs_RBF_FS    1.457e-02  3.856e-01
AE_FE_vs_VAE_FS    1.052e-07  1.864e-59
AE_FE_vs_DEC_FS    1.645e-23  8.650e-15
RBF_FC_vs_VAE_FC   4.838e-02  9.929e-15
RBF_FC_vs_DEC_FC   3.959e-32  2.827e-02
RBF_FC_vs_AE_FS    5.051e-14  8.868e-01
RBF_FC_vs_RBF_FS   4.831e-08  7.610e-02
RBF_FC_vs_VAE_FS   8.910e-01  1.508e-43
RBF_FC_vs_DEC_FS   2.003e-26  9.743e-04
VAE_FC_vs_DEC_FC   6.090e-35  4.334e-03
VAE_FC_vs_AE_FS    5.783e-07  3.582e-09
VAE_FC_vs_RBF_FS   2.303e-10  5.168e-24
VAE_FC_vs_VAE_FS   3.626e-02  7.704e-01
VAE_FC_vs_DEC_FS   6.997e-30  1.528e-01
DEC_FC_vs_AE_FS    1.822e-02  6.385e-01
DEC_FC_vs_RBF_FS   3.221e-03  8.584e-01
DEC_FC_vs_VAE_FS   5.010e-30  2.009e-27
DEC_FC_vs_DEC_FS   3.229e-01  1.309e-03
AE_FS_vs_RBF_FS    7.886e-02  1.861e-01
AE_FS_vs_VAE_FS    4.348e-08  4.993e-32


In [184]:
file_name = "Summary"
keys_ae = list(comp_df_ae.keys())
keys_rbf = list(comp_df_rbf.keys())
keys_vae = list(comp_df_vae.keys())
keys_dec = list(comp_df_dec.keys())

x = [k for k in keys_ae for i in range(2)] + \
    [k for k in keys_rbf for i in range(2)] + \
    [k for k in keys_vae for i in range(2)] + \
    [k for k in keys_dec for i in range(2)]

y = []
for k in keys_ae:
    y += [comp_df_ae[k][metric][0][0] for metric in METRICS]
for k in keys_rbf:
    y += [comp_df_rbf[k][metric][0][0] for metric in METRICS]
for k in keys_vae:
    y += [comp_df_vae[k][metric][0][0] for metric in METRICS]
for k in keys_dec:
    y += [comp_df_dec[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys_ae)*4)
fig = plot_bar(results=df_formatted, title="Summary - Transferred models compared to the reference perf. of AE+MoE", vertical=True,
        width=600, height=600, add_hline=False, annotation_text="Reference perf. of AE+MoE", annotation_position="top right")
save_plot(fig, "img/"+file_name+".svg")
fig.show()


In [185]:
for k in keys_ae:
    for metric in METRICS:
        comp_df_ae[k][metric]= comp_df_ae[k][metric]= "{:0.3e}".format(comp_df_ae[k][metric][1][0])
        
for k in keys_rbf:
    for metric in METRICS:
        comp_df_rbf[k][metric]= comp_df_rbf[k][metric]= "{:0.3e}".format(comp_df_rbf[k][metric][1][0])
for k in keys_vae:
    for metric in METRICS:
        comp_df_vae[k][metric]= comp_df_vae[k][metric]= "{:0.3e}".format(comp_df_vae[k][metric][1][0])
for k in keys_dec:
    for metric in METRICS:
        comp_df_dec[k][metric]= comp_df_dec[k][metric]= "{:0.3e}".format(comp_df_dec[k][metric][1][0])
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_ae = pd.DataFrame(comp_df_ae)
comp_pd_rbf = pd.DataFrame(comp_df_rbf)
comp_pd_vae = pd.DataFrame(comp_df_vae)
comp_pd_dec = pd.DataFrame(comp_df_dec)

print(comp_pd_ae)
print(comp_pd_rbf)
print(comp_pd_vae)
print(comp_pd_dec)
comp_pd_ae.to_csv("tables/"+file_name+"_ae.csv")
comp_pd_rbf.to_csv("tables/"+file_name+"_rbg.csv")
comp_pd_vae.to_csv("tables/"+file_name+"_vae.csv")
comp_pd_dec.to_csv("tables/"+file_name+"_dec.csv")

               AE_{FE}    AE_{FS}
recon_error  1.901e-13  1.753e-14
delta_rot    3.736e-01  9.530e-02
              RBF_{FC}   RBF_{FS}
recon_error  1.904e-29  2.871e-10
delta_rot    7.146e-03  4.482e-01
              VAE_{FC}   VAE_{FS}
recon_error  7.332e-12  4.278e-29
delta_rot    1.308e-12  1.577e-31
              DEC_{FC}   DEC_{FS}
recon_error  1.660e-50  5.146e-48
delta_rot    5.156e-01  2.371e-05


In [227]:
METRICS = ["recon_error","adv_error", "rot_error", "delta_rot"]
METRICS_RENAME = ["Re. error", "Adv. error", "Rot. error", "RMS"]
keys = ["AE","RBF-IN","RBF-CAT","VAE-IN","VAE-CAT", "DEC-IN", "DEC-CAT"]
results = {}
for k in keys:
    k1 = k + "+MoE"
    k2 = k + "+LSTM"
    p1 = pJoin(REF_SAVE_PATH,k1+".pbz2")
    p2 = pJoin(REF_SAVE_PATH,k2+".pbz2")
    if os.path.exists(p1):
        f1 = load(p1)
        for i in range(len(f1[METRICS[0]])):
            results[k1+"_R"+str(i+1)] = {}
            for metric,me in zip(METRICS,METRICS_RENAME):
                results[k1+"_R"+str(i+1)][me] = np.mean(f1[metric][i])
    if os.path.exists(p2):
        f2 = load(p2)
        for i in range(len(f2[METRICS[0]])):
            results[k2+"_R"+str(i+1)] = {}
            for metric,me in zip(METRICS,METRICS_RENAME):
                results[k2+"_R"+str(i+1)][me] = np.mean(f2[metric][i])


In [228]:
df = pd.DataFrame(results).T
df.to_csv("tables/ref_res_all.csv")

In [229]:
keys = ["AE","RBF-IN","RBF-CAT","VAE-IN","VAE-CAT", "DEC-IN", "DEC-CAT"]
SUFFIX = {"Raw":"_RAW", "Frozen":"_F", "Trained":"_T"}

def insert(results,k1,p1):
    f1 = load(p1)
    for i in range(len(f1[METRICS[0]])):
        results[k1+"_R"+str(i+2)] = {}
        for metric,me in zip(METRICS,METRICS_RENAME):
            results[k1+"_R"+str(i+2)][me] = np.mean(f1[metric][i])

results = {}
for k in keys:
    k1 = k + "+MoE" + "_RAW"
    k2 = k + "+MoE" + "_F"
    k3 = k + "+MoE" + "_T"

    p1 = pJoin(TRANS_SAVE_PATH,k1+".pbz2")
    p2 = pJoin(TRANS_SAVE_PATH,k2+".pbz2")
    p3 = pJoin(TRANS_SAVE_PATH,k3+".pbz2")

    if os.path.exists(p1):
        insert(results,k1,p1)
    if os.path.exists(p2):
        insert(results,k2,p2)
    if os.path.exists(p3):
        insert(results,k3,p3)



In [230]:
df = pd.DataFrame(results).T
df.to_csv("tables/trans_res_all.csv")

In [231]:
keys = ["AE","RBF-IN","RBF-CAT","VAE-IN","VAE-CAT", "DEC-IN", "DEC-CAT"]
results = {}
for k in keys:
    k1 = k + "+MoE" + "_RAW"
    k2 = k + "+MoE" + "_F"
    k3 = k + "+MoE" + "_T"

    p1 = pJoin(REDUC_SAVE_PATH,k1+".pbz2")
    p2 = pJoin(REDUC_SAVE_PATH,k2+".pbz2")
    p3 = pJoin(REDUC_SAVE_PATH,k3+".pbz2")

    if os.path.exists(p1):
        insert(results,k1,p1)
    if os.path.exists(p2):
        insert(results,k2,p2)
    if os.path.exists(p3):
        insert(results,k3,p3)


In [232]:
df = pd.DataFrame(results).T
df.to_csv("tables/reduc_res_all.csv")